In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
import talib as ta

In [2]:
D = pd.read_parquet('btcusd_hourly_price.parquet', engine='pyarrow')
D.head(10)

,Open,High,Low,Close,Volume,return_forward
Timestamp,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 15:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 16:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 17:00:00,4.58,4.58,4.58,4.58,0.0,0.000000
2012-01-01 18:00:00,4.58,4.58,4.58,4.58,0.0,0.000000


# Cleaning

In [3]:
# D = D.set_index('Open Time')
# D.index.name = 'Date'
# D.drop(['Close Time', 'Ignore'], axis=1, inplace=True)
# D

In [4]:
D.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume' : 'volume'}, inplace=True)
D

,open,high,low,close,volume,return_forward
Timestamp,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142


In [5]:
for col in D.columns:
    print(f'{col} is {D[col].isna().sum()}')

open is 1607
high is 1607
low is 1607
close is 1607
volume is 0
return_forward is 0


In [6]:
for col in D.columns:
    D[col] = D[col].fillna(method='ffill')


C:\Users\User\AppData\Local\Temp\ipykernel_22112\2569728342.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  D[col] = D[col].fillna(method='ffill')


In [7]:
for col in D.columns:
    print(f'{col} is {D[col].isna().sum()}')

open is 0
high is 0
low is 0
close is 0
volume is 0
return_forward is 0


In [8]:
del D['return_forward']
D

,open,high,low,close,volume
Timestamp,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000
...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950


In [9]:
D['return_forward'] = D['close'].pct_change().shift(-1)
D

,open,high,low,close,volume,return_forward
Timestamp,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142


In [10]:
D.iloc[-1,5] = 0.000150

D

,open,high,low,close,volume,return_forward
Timestamp,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000
...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142


In [11]:
D['return'] = D['close'].pct_change()
D

,open,high,low,close,volume,return_forward,return
Timestamp,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858


# Volume

In [12]:
D['obv'] = ta.OBV(D['close'],D['volume'])

D

,open,high,low,close,volume,return_forward,return,obv
Timestamp,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280


# Trend

## MA

In [13]:
ma_value = [5,10,20,50,100,200]

for val in ma_value:
    D[f'sma_{val}'] = D['close'].rolling(window=val).mean()

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,sma_20,sma_50,sma_100,sma_200
Timestamp,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,94002.90,93690.04,93984.71,95406.805
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,93998.40,93688.50,93978.86,95392.940
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,94007.25,93693.52,93974.09,95387.555


## EMA

In [14]:
ma_value = [5,10,20,50,100,200]

for val in ma_value:
    D[f'ema_{val}'] = D['close'].ewm(span=val, adjust=False).mean()

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,sma_20,sma_50,sma_100,sma_200,ema_5,ema_10,ema_20,ema_50,ema_100,ema_200
Timestamp,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,NaN,NaN,NaN,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,94002.90,93690.04,93984.71,95406.805,94208.917308,94272.313800,94116.667231,94001.883659,94431.560261,95488.516399
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,93998.40,93688.50,93978.86,95392.940,93944.278205,94116.438564,94049.841781,93978.868614,94411.430355,95467.884395
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,94007.25,93693.52,93974.09,95387.555,93856.852137,94037.449734,94014.809230,93967.226707,94396.986189,95450.114401


## MACD

In [15]:
D['macd_12_26'], D['signal_line_9'], D['macd_histogram'] = ta.MACD(D['close'],12,26)

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,sma_200,ema_5,ema_10,ema_20,ema_50,ema_100,ema_200,macd_12_26,signal_line_9,macd_histogram
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,NaN,4.580000,4.580000,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,95406.805,94208.917308,94272.313800,94116.667231,94001.883659,94431.560261,95488.516399,201.771394,223.884683,-22.113289
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,95392.940,93944.278205,94116.438564,94049.841781,93978.868614,94411.430355,95467.884395,120.497438,203.207234,-82.709795
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,95387.555,93856.852137,94037.449734,94014.809230,93967.226707,94396.986189,95450.114401,76.747218,177.915231,-101.168013


# Momentum

## ADX

In [16]:
D['adx'] = ta.ADX(D['high'], D['low'], D['close'],14)
D['+di'] = ta.PLUS_DI(D['high'], D['low'], D['close'],14)
D['-di'] = ta.MINUS_DI(D['high'], D['low'], D['close'],14)

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,ema_20,ema_50,ema_100,ema_200,macd_12_26,signal_line_9,macd_histogram,adx,+di,-di
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,4.580000,4.580000,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,94116.667231,94001.883659,94431.560261,95488.516399,201.771394,223.884683,-22.113289,40.943650,24.672194,28.601038
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,94049.841781,93978.868614,94411.430355,95467.884395,120.497438,203.207234,-82.709795,38.995557,22.464248,29.578700
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,94014.809230,93967.226707,94396.986189,95450.114401,76.747218,177.915231,-101.168013,37.186614,21.273029,28.010220


## RSI

In [17]:
list = [7, 14]

for val in list:
    D[f'rsi_{val}'] = ta.RSI(D['close'],val)

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,ema_100,ema_200,macd_12_26,signal_line_9,macd_histogram,adx,+di,-di,rsi_7,rsi_14
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,4.580000,4.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,94431.560261,95488.516399,201.771394,223.884683,-22.113289,40.943650,24.672194,28.601038,44.656824,49.600999
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,94411.430355,95467.884395,120.497438,203.207234,-82.709795,38.995557,22.464248,29.578700,34.760191,42.325843
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,94396.986189,95450.114401,76.747218,177.915231,-101.168013,37.186614,21.273029,28.010220,41.563036,46.155291


# Volatility

## Average True Range

In [18]:
list = [7,14,21]

for val in list:
    D[f'atr_{val}'] = ta.ATR(D['high'], D['low'],D['close'],val)

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,signal_line_9,macd_histogram,adx,+di,-di,rsi_7,rsi_14,atr_7,atr_14,atr_21
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,223.884683,-22.113289,40.943650,24.672194,28.601038,44.656824,49.600999,787.401340,594.021005,490.327871
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,203.207234,-82.709795,38.995557,22.464248,29.578700,34.760191,42.325843,783.344006,605.805219,503.121782
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,177.915231,-101.168013,37.186614,21.273029,28.010220,41.563036,46.155291,734.437720,594.033418,500.163602


## Bollinger Bands

In [19]:
D['upper_band_10'], D['middle_band_10'], D['lower_band_10'] = ta.BBANDS(D['close'], timeperiod=10, nbdevup=2, nbdevdn=2, matype=0)
D['upper_band_20'], D['middle_band_20'], D['lower_band_20'] = ta.BBANDS(D['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,rsi_14,atr_7,atr_14,atr_21,upper_band_10,middle_band_10,lower_band_10,upper_band_20,middle_band_20,lower_band_20
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,49.600999,787.401340,594.021005,490.327871,95801.067876,94516.7,93232.332124,95433.425204,94002.90,92572.374796
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,42.325843,783.344006,605.805219,503.121782,95876.395307,94461.6,93046.804693,95435.711991,93998.40,92561.088009
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,46.155291,734.437720,594.033418,500.163602,95898.271810,94421.0,92943.728190,95434.444994,94007.25,92580.055006


# Columns

In [20]:
D.columns

Index(['open', 'high', 'low', 'close', 'volume', 'return_forward', 'return',
       'obv', 'sma_5', 'sma_10', 'sma_20', 'sma_50', 'sma_100', 'sma_200',
       'ema_5', 'ema_10', 'ema_20', 'ema_50', 'ema_100', 'ema_200',
       'macd_12_26', 'signal_line_9', 'macd_histogram', 'adx', '+di', '-di',
       'rsi_7', 'rsi_14', 'atr_7', 'atr_14', 'atr_21', 'upper_band_10',
       'middle_band_10', 'lower_band_10', 'upper_band_20', 'middle_band_20',
       'lower_band_20'],
      dtype='object')

# Export

In [21]:
D.to_parquet('btcusd_hourly_price_indicators.parquet')

In [22]:
D = pd.read_parquet('btcusd_hourly_price_indicators.parquet', engine='pyarrow')
D

,open,high,low,close,volume,return_forward,return,obv,sma_5,sma_10,...,rsi_14,atr_7,atr_14,atr_21,upper_band_10,middle_band_10,lower_band_10,upper_band_20,middle_band_20,lower_band_20
Timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.000000,0.000000,0.000000,0.000000,4.58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,93760.00,94158.00,93600.00,94008.00,7.863550,-0.006308,0.002666,175364.391806,94342.20,94516.7,...,49.600999,787.401340,594.021005,490.327871,95801.067876,94516.7,93232.332124,95433.425204,94002.90,92572.374796
2024-12-31 20:00:00,94025.00,94059.00,93300.00,93415.00,4.565475,0.002858,-0.006308,175359.826331,93968.20,94461.6,...,42.325843,783.344006,605.805219,503.121782,95876.395307,94461.6,93046.804693,95435.711991,93998.40,92561.088009
2024-12-31 21:00:00,93386.00,93777.00,93336.00,93682.00,28.484950,-0.004142,0.002858,175388.311280,93768.80,94421.0,...,46.155291,734.437720,594.033418,500.163602,95898.271810,94421.0,92943.728190,95434.444994,94007.25,92580.055006
